In [1]:
import warnings
warnings.simplefilter('ignore')

import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model,Sequential
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from tensorflow.python.keras.layers import Flatten, Dense, Dropout,BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense


2024-06-06 13:47:15.006372: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
batch_size,img_height,img_width = 8, 256, 256

data_dir='/Users/pratham/Desktop/Project/Algorithms/Kidney_Disease/dataset'


train=tf.keras.utils.image_dataset_from_directory(data_dir,image_size=(img_height,img_width),
                                                validation_split=0.1,
                                                subset='training',seed=123,shuffle=True)

val=tf.keras.utils.image_dataset_from_directory(data_dir,image_size=(img_height,img_width),
                                                validation_split=0.1,
                                                subset='validation',seed=123,shuffle=True)

Found 9519 files belonging to 4 classes.
Using 8568 files for training.
Found 9519 files belonging to 4 classes.
Using 951 files for validation.


In [3]:
label_to_class_name = dict(zip(range(len(train.class_names)), train.class_names))
label_to_class_name

{0: 'Cyst', 1: 'Normal', 2: 'Stone', 3: 'Tumor'}

#### CNN

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(4, activation=tf.nn.softmax)
])

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
history = model.fit(train,validation_data=val,epochs=15)


Epoch 1/15
268/268 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.7361 - loss: 107.5345 - val_accuracy: 0.9989 - val_loss: 0.0049
Epoch 2/15
268/268 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.9997 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 1.6823e-04
Epoch 3/15
268/268 ━━━━━━━━━━━━━━━━━━━━ 288s 1s/step - accuracy: 1.0000 - loss: 9.6872e-05 - val_accuracy: 1.0000 - val_loss: 8.2501e-05
Epoch 4/15
268/268 ━━━━━━━━━━━━━━━━━━━━ 290s 1s/step - accuracy: 1.0000 - loss: 3.1854e-05 - val_accuracy: 1.0000 - val_loss: 4.1689e-05
Epoch 5/15
268/268 ━━━━━━━━━━━━━━━━━━━━ 287s 1s/step - accuracy: 1.0000 - loss: 1.6967e-05 - val_accuracy: 1.0000 - val_loss: 2.5730e-05
Epoch 6/15
268/268 ━━━━━━━━━━━━━━━━━━━━ 287s 1s/step - accuracy: 1.0000 - loss: 1.0166e-05 - val_accuracy: 1.0000 - val_loss: 1.7984e-05
Epoch 7/15
268/268 ━━━━━━━━━━━━━━━━━━━━ 285s 1s/step - accuracy: 1.0000 - loss: 6.6942e-06 - val_accuracy: 1.0000 - val_loss: 1.3437e-05
Epoch 8/15
268/268 ━━━━━━━━━━━━━━━━━━━━ 287s 1s/ste

In [6]:
model.save("../../Models/KidneyDisease_prediction.h5")


In [7]:
img_height,img_width = 256, 256

import tensorflow as tf

# Define the path to the test image
test_image_path = '/Users/pratham/Desktop/Project/Algorithms/Kidney_Disease/testt/Tumor- (1).jpg'

# Preprocess the image
img = tf.keras.preprocessing.image.load_img(test_image_path, target_size=(img_height, img_width))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(4, activation=tf.nn.softmax)
])

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.load_weights("kidney_disease_prediction.h5")

# Load your trained model
# model = tf.keras.models.load_weights('kidney_disease_prediction.h5')

# Make predictions
predictions = model.predict(img_array)
predicted_class = predictions.argmax(axis=-1)[0]

print("Predicted class:", predicted_class)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Predicted class: 3


In [8]:
y_true_cnn = []
y_pred_cnn = []
for images, labels in val:
    predictions = model.predict(images)
    y_true_cnn.extend(labels.numpy())
    y_pred_cnn.extend(np.argmax(predictions, axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 409ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 625ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 590ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 586ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 511ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 610ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 631ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 657ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

2024-06-06 15:01:45.022397: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [9]:
cnn_accuracy = accuracy_score(y_true_cnn, y_pred_cnn)
cnn_precision = precision_score(y_true_cnn, y_pred_cnn, average='weighted')
cnn_recall = recall_score(y_true_cnn, y_pred_cnn, average='weighted')
cnn_f1 = f1_score(y_true_cnn, y_pred_cnn, average='weighted')

print("CNN Model Evaluation:")
print("Accuracy:", cnn_accuracy)
print("Precision:", cnn_precision)
print("Recall:", cnn_recall)
print("F1 Score:", cnn_f1)

CNN Model Evaluation:
Accuracy: 0.2618296529968454
Precision: 0.2874400252986491
Recall: 0.2618296529968454
F1 Score: 0.18856525418909378


In [10]:
cnn_metrics = {
    "Model": ["CNN"],
    "Accuracy": [cnn_accuracy],
    "Precision": [cnn_precision],
    "Recall": [cnn_recall],
    "F1 Score": [cnn_f1]
}
cnn_df = pd.DataFrame(cnn_metrics)
cnn_df

,Model,Accuracy,Precision,Recall,F1 Score
0,CNN,0.26183,0.28744,0.26183,0.188565
